In [1]:
%load_ext autoreload
%autoreload 2
from individual_channel_gradcam import GradCAM
import os
import cv2 
from model import EfficientnetWithFinetuning
from data import MelanomaData
import os
import matplotlib.pyplot as plt
import torch as t
import numpy as np
import seaborn as sns

In [2]:
markers = ['ADAM10', 'Bcl-2', 'CD10', 'CD107a', 'CD13', 'CD138', 'CD14', 'CD1a', 'CD2', 'CD25', 'CD271', 'CD3', 'CD36', 'CD4', 'CD44', 'CD45', 'CD45RA', 'CD45RO', 'CD5', 'CD56', 'CD6', 'CD63', 'CD66abce', 'CD7', 'CD71', 'CD8', 'CD9', 'CD95', 'Collagen IV', 'Cytokeratin-14', 'EBF-P', 'EGFR', 'EGFR-AF488', 'HLA-ABC', 'HLA-DR', 'KIP1', 'Ki67', 'L302', 'MCSP', 'Melan-A', 'Nestin-AF488', 'Notch-1', 'Notch-3', 'PPARgamma', 'PPB', 'RIM3', 'TAP73', 'Vimentin', 'p63', 'phospho-Connexin']    

In [3]:
model = EfficientnetWithFinetuning(indim=len(markers))
model.load_state_dict(t.load("/data_nfs/je30bery/melanoma_data/model/saved_models/model_2023-11-20 16:27:56.558630_f1=0.9260606060606061_acc=0.9166666666666666_11.pt"))
for l in model.features:
    l.requires_grad = True
model.eval()

/data/bionets/je30bery/anaconda3/envs/als_analysis/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


EfficientnetWithFinetuning(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(50, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): 

In [4]:
target_layer = 0
gradcam = GradCAM(model.cuda(), target_layer)

In [5]:
data = "/data_nfs/datasets/melc/melanoma/processed"

In [6]:
vdata = sorted([os.path.join(data, "validation", d) for d in sorted(os.listdir(os.path.join(data, "validation")))] + [os.path.join(data, "training", d) for d in sorted(os.listdir(os.path.join(data, "training")))]) # balance_and_shuffle(os.path.join(data, "validation"))#
vdl = t.utils.data.DataLoader(MelanomaData(markers, vdata, mode="val"), batch_size=10) #, shuffle=True)
it = iter(vdl)

In [ ]:
val_split = sorted([os.path.join("/data_nfs/datasets/melc/melanoma/processed", "validation", d) for d in sorted(os.listdir(os.path.join("/data_nfs/datasets/melc/melanoma/processed", "validation")))]

In [7]:
len(vdata)

197

In [ ]:
cams = list()
while True:
    try:
        input_t, label = next(it)
    except StopIteration:
        break
    gc = gradcam.cam(input_t.cuda(), label.cuda())    
    cams.append(gc)

/data/bionets/je30bery/anaconda3/envs/als_analysis/lib/python3.9/site-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [ ]:
len(vdata)

In [ ]:
markers = np.array(markers)
signi_markers = list()
for i in range(4):
    test = cams[i].cpu().numpy()
    for j in range(test.shape[0]):
        sum = np.sum(test[j] * (test[j] > 0.05) , axis=(1, 2))
        where = np.where(sum)
        signi_markers.append(markers[where])
        print(markers[where])

In [ ]:
sign, counts = np.unique(np.concatenate(signi_markers), return_counts=True)

In [ ]:
counts[np.where(counts>=10)]

In [ ]:
count_sort_ind = np.argsort(-counts[np.where(counts>=10)])
sign[np.where(counts>=10)][count_sort_ind]

In [ ]:
counts[count_sort_ind]

In [ ]:
plt.imshow(cv2.imread("/data_nfs/datasets/melc/melanoma/processed/validation/Melanoma_36_202009111023_3/21100_TAP73-FITC_5000_068.tif", cv2.IMREAD_GRAYSCALE))

In [ ]:
import torch as t
#from pytorch_grad_cam import GradCAM
#from pytorch_grad_cam.utils.image import show_cam_on_image
#from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import cv2 
from model import EfficientnetWithFinetuning
from data import MelanomaData
import os
import numpy as np
from melanoma_main import balance_and_shuffle
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
plt.style.use('dark_background')
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
target_layers = [model.features[0]]
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

In [ ]:
if 0:
    h, w = 13, 3
    f, axs = plt.subplots(h, w, figsize=(15, 50))
    for idx in range(len(vdata)):
        axs[idx % h, int(idx / h)].imshow(visualizations[idx])
        title = os.path.basename(vdata[idx]) #"Melanoma" if labels[idx] == 0 else "Nevi"
        axs[idx % h, int(idx / h)].set_title(title)
        axs[idx % h, int(idx / h)].axis("off")
    plt.savefig("gradcam_val.pdf")
    plt.show()

In [ ]:
roi_data = {
    '202003251552_1': 20974,
    '202004091556_1': 20982,
    '202004091556_4': 20985,
    '202004221438_1': 20990,
    '202004221438_3': 20992,
    '202004221438_4': 20993,
    '202004301203_2': 20998,
    '202004301203_4': 21000,
    '202005071402_4': 21010,
    '202007020851_4': 21047,
    '202007211656_4': 21060,
    '202007281047_3': 21069,
    '202009031055_3': 21087,
    '202009111023_3': 21100,
    '202009181022_1': 21102,
    '202009181022_3': 21104,
    '202010211103_1': 21124,
    '202010211055_2': 21129,
}

In [ ]:
path_dict = dict()
for roi in roi_data:
    print(roi)
    path = [p for p in vdata if roi in p]
    print(path)
    assert len(path) == 1
    path = path[0]
    idx = vdata.index(path)
    roi_path = f"/data_nfs/je30bery/melanoma_data/ROI/{roi_data[roi]}MelanomaArea.tif"
    path_dict[idx] = roi_path 

In [ ]:
len(path_dict)

In [ ]:
f, axs = plt.subplots(3, 6, figsize=(18,9))
for i, k in enumerate(path_dict):
    boundaries = cv2.resize((cv2.imread(path_dict[k], cv2.IMREAD_GRAYSCALE) > 0).astype(np.uint8), (512,512))
    axs[int(i/6), i%6].imshow(mark_boundaries(visualizations[k], boundaries, color=(1,0,0)))
    axs[int(i/6), i%6].axis("off")
    axs[int(i/6), i%6].set_title(k)
plt.suptitle("GradCAM results with marked expert annotations")
plt.tight_layout()
plt.savefig("gradcam_val.pdf")

In [ ]:
grayscale_cam.shape